In [33]:
import import_ipynb
from spider import spider
import pandas as pd

In [34]:
file_path = 'datasets/glass1.dat'
with open(file_path, 'r') as file:
    lines = file.readlines()
    data_line_index = next((i for i, line in enumerate(lines) if line.startswith('@data')), None)
df = pd.read_csv('datasets/glass1.dat', delimiter=',', skiprows=range(data_line_index))
df = df.reset_index()
df['@data'] = df['@data'].map({' positive': 1, ' negative': 0})
df.rename(columns={'@data': 'labels'}, inplace=True)
df

,level_0,level_1,level_2,level_3,level_4,level_5,level_6,level_7,level_8,labels
0,1.515888,12.87795,3.43036,1.40066,73.2820,0.68931,8.04468,0.00000,0.1224,0
1,1.517642,12.97770,3.53812,1.21127,73.0020,0.65205,8.52888,0.00000,0.0000,0
2,1.522130,14.20795,3.82099,0.46976,71.7700,0.11178,9.57260,0.00000,0.0000,0
3,1.522221,13.21045,3.77160,0.79076,71.9884,0.13041,10.24520,0.00000,0.0000,0
4,1.517551,13.39000,3.65935,1.18880,72.7892,0.57132,8.27064,0.00000,0.0561,0
...,...,...,...,...,...,...,...,...,...,...
209,1.517278,14.70005,0.00000,2.34119,73.2820,0.00000,8.94852,0.66150,0.0000,0
210,1.516230,14.14145,0.00000,2.88047,72.6100,0.08073,9.18524,1.06155,0.0000,0
211,1.518371,14.32100,3.25974,2.21921,71.2492,1.45935,5.78508,1.62855,0.0000,0
212,1.516572,14.79980,0.00000,1.99130,73.1084,0.00000,8.28140,1.71045,0.0000,0


In [35]:
X = df.drop('labels', axis=1).values
X.shape

(214, 9)

In [36]:
y = df['labels'].values
y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64)

In [37]:
new_X, new_y = spider(X, y, amplification_type='week_amplification')
new_y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1], dtype=int64)

In [42]:
import numpy as np
from sklearn.model_selection import cross_val_score, KFold
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import (recall_score, accuracy_score, balanced_accuracy_score, 
                             precision_score, f1_score, confusion_matrix)


def claculate_metrics(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    recall = recall_score(y_true, y_pred)
    specificity = tn / (tn+fp)
    accuracy = accuracy_score(y_true, y_pred)
    balanced_accuracy = balanced_accuracy_score(y_true, y_pred)
    geometric_mean = np.sqrt(tp*tn)
    precision = precision_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)

    metrics = {
        'recall': recall,
        'specificity': specificity,
        'accuracy': accuracy,
        'balanced_accuracy': balanced_accuracy,
        'geometric_mean': geometric_mean,
        'precision': precision,
        'f1': f1,
    }
    return metrics


def test_models(X, y):
    classifiers = [MultinomialNB(), KNeighborsClassifier(), DecisionTreeClassifier()]
    kf = KFold(n_splits=5)

    for classifier in classifiers:
        fold_metrics = []
        for train_index, test_index in kf.split(X):
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]
            classifier.fit(X_train, y_train)
            try: 
                y_pred = classifier.predict(X_test)
                fold_metrics.append(claculate_metrics(y_test, y_pred))
            except:
                continue
        print(f"{classifier.__class__.__name__}")
        avg_metrics = {metric: np.mean([fold[metric] for fold in fold_metrics]) for metric in fold_metrics[0]}
        for metric, value in avg_metrics.items():
            print(f"{metric}: {value}")
        print("\n")

In [43]:
test_models(X, y)

MultinomialNB
recall: 0.0125
specificity: nan
accuracy: 0.2894795127353267
balanced_accuracy: 0.2471241990137339
geometric_mean: 1.0198039027185568
precision: 0.1
f1: 0.02222222222222222


KNeighborsClassifier
recall: 0.3006326949384405
specificity: nan
accuracy: 0.6016611295681064
balanced_accuracy: 0.5849994077336759
geometric_mean: 5.757021148285399
precision: 0.5022222222222222
f1: 0.36753278688524593


DecisionTreeClassifier
recall: 0.33712380300957595
specificity: nan
accuracy: 0.526578073089701
balanced_accuracy: 0.5242925618866112
geometric_mean: 5.0036855035053
precision: 0.39473684210526316
f1: 0.3503591903591903




c:\Users\rober\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\rober\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:2006: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
C:\Users\rober\AppData\Local\Temp\ipykernel_16884\1977209671.py:13: RuntimeWarning: invalid value encountered in longlong_scalars
  specificity = tn / (tn+fp)
c:\Users\rober\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:2006: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
c:\Users\rober\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\

In [44]:
test_models(new_X, new_y)

MultinomialNB
recall: 0.12240896358543418
specificity: 0.453234693877551
accuracy: 0.2226938775510204
balanced_accuracy: 0.3224748899559824
geometric_mean: 1.3843251395565515
precision: 0.21428571428571427
f1: 0.12503293807641633


KNeighborsClassifier
recall: 0.3691230338289162
specificity: 0.7644693877551021
accuracy: 0.6631020408163264
balanced_accuracy: 0.7001023332409886
geometric_mean: 8.35971046405668
precision: 0.5553320683111954
f1: 0.4203519788482194


DecisionTreeClassifier
recall: 0.4649859943977591
specificity: 0.7521224489795919
accuracy: 0.7371428571428571
balanced_accuracy: 0.7439011604641858
geometric_mean: 9.424816095585927
precision: 0.5542650664601884
f1: 0.5005633473585788




c:\Users\rober\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\rober\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:2006: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
c:\Users\rober\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\rober\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:2006: UserWarning: y_pred contains cla

In [46]:
new_X, new_y = spider(X, y, amplification_type='week_amplification_with_relabeling')
test_models(new_X, new_y)

MultinomialNB
recall: 0.582051282051282
specificity: 0.44226190476190474
accuracy: 0.2770196078431372
balanced_accuracy: 0.5121565934065934
geometric_mean: 3.319513863747408
precision: 0.4752798573975044
f1: 0.22552137188829277


KNeighborsClassifier
recall: 0.8009324009324009
specificity: 0.7691197691197692
accuracy: 0.6984313725490197
balanced_accuracy: 0.7850260850260851
geometric_mean: 12.747080097743616
precision: 0.6155718079317399
f1: 0.5444055436433486


DecisionTreeClassifier
recall: 0.8954545454545455
specificity: 0.6727813852813853
accuracy: 0.7393725490196078
balanced_accuracy: 0.7841179653679655
geometric_mean: 12.97057647409531
precision: 0.5943695014662757
f1: 0.6025845947312684




In [47]:
new_X, new_y = spider(X, y, amplification_type='strong_amplification')
test_models(new_X, new_y)

MultinomialNB
recall: 0.9785164835164835
specificity: nan
accuracy: 0.632973005255614
balanced_accuracy: 0.6179212454212455
geometric_mean: 3.5
precision: 0.6462393803098451
f1: 0.7429788810410176


KNeighborsClassifier
recall: 0.8993859082094374
specificity: nan
accuracy: 0.7452938365981845
balanced_accuracy: 0.7830470999588648
geometric_mean: 15.884240319942368
precision: 0.7875
f1: 0.7985857323461192


DecisionTreeClassifier
recall: 1.0
specificity: 0.4843073593073593
accuracy: 0.7028985507246377
balanced_accuracy: 0.7421536796536796
geometric_mean: 28.15264493789817
precision: 0.6482517132553259
f1: 0.7557687183586465




C:\Users\rober\AppData\Local\Temp\ipykernel_16884\1977209671.py:13: RuntimeWarning: invalid value encountered in longlong_scalars
  specificity = tn / (tn+fp)
c:\Users\rober\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:2006: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
C:\Users\rober\AppData\Local\Temp\ipykernel_16884\1977209671.py:13: RuntimeWarning: invalid value encountered in longlong_scalars
  specificity = tn / (tn+fp)
c:\Users\rober\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:2006: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
C:\Users\rober\AppData\Local\Temp\ipykernel_16884\1977209671.py:13: RuntimeWarning: invalid value encountered in longlong_scalars
  specificity = tn / (tn+fp)
c:\Users\rober\AppData\Local\Programs\Python\Python310\lib\site-packages\sklear